
* cd some_directory
* git clone https://github.com/PHAREHUB/PHARE
* cd where_is_this_notebook


In [1]:
# some preliminary settings and functions
import sys
sys.path.append("/home/tavant/PHARE/pyphare/") # <<<<======= replace /home/aunai with where you have cloned phare
import os
import numpy as np
import pyphare
from pyphare.pharesee.hierarchy import finest_field
from pyphare.pharesee.hierarchy import get_times_from_h5
from pyphare.pharesee.hierarchy import hierarchy_from
from pyphare.pharesee.plotting import zoom_effect
import matplotlib.pyplot as plt
from pyphare.pharesee.run import Run
%matplotlib widget

In [2]:
path = "/DATA/tavant/2strRefin1000/"

In [3]:
# get all dump times for this file
# this file contains inner patch domain particles for all the "main" population
# that's the same dump time for the beam anyway
times= get_times_from_h5(path+"ions_pop_main_domain.h5")

In [4]:
# create the run object from which we'll get data
run = Run(path)

In [5]:
# we have the "main" and "beam" populations in that run
# read both and make a patch hierarchy at t=85
ions = run.GetParticles(times[1700], ["main", "beam"])

In [6]:
def get_parts(ions, **kwargs):
    """
        return the selectred parts
    """
    import copy

    
    usr_lvls = kwargs.get("levels",(0,))
    finest = kwargs.get("finest", False)
    pops = kwargs.get("pop",[])
    time = kwargs.get("time", ions.times()[0])
    all_pops = list(ions.level(0,time).patches[0].patch_datas.keys())

    if finest:
        final = finest_part_data(self)

    else:
        final = {pop:None for pop in all_pops}
        for lvl_nbr,level in ions.levels(time).items():
            if lvl_nbr not in usr_lvls:
                continue
            for ip, patch in enumerate(level.patches):
                if len(pops)==0:
                    pops = list(patch.patch_datas.keys())

                for pop in pops:
                    tmp = copy.copy(patch.patch_datas[pop].dataset)

                    if final[pop] is None:
                        final[pop] = tmp
                    else:
                        final[pop].add(tmp)

    # select particles
    if "select" in kwargs:
        for pop, particles in final.items():
            final[pop] = kwargs["select"](particles)
            
    return final

In [7]:


# then plot the freaking phase space

xlim=(0, 33)
ylim=(-1.5, 5.5)

fig, ax = plt.subplots(figsize=(10,4))

part, plot = ions.dist_plot(axis=("x", "Vx"),
                   ax=ax,
                   finest=True,              # take finest particles where available
                   #sigma=(1,1),             # gaussian filter sigmas
                   color_max=0.1,
                   xlim=xlim,
                   ylim=ylim,
                   vmin=ylim[0],
                   vmax=ylim[1],
                   title="Streaming instability",
                   xlabel="X - Position",
                   ylabel= "Vx - Velocity"
                  )
ax.vlines(10, ylim[0], ylim[1], color="k")
ax.vlines(14, ylim[0], ylim[1], color="r")
ax.vlines(18, ylim[0], ylim[1], color="r")
ax.vlines(22, ylim[0], ylim[1], color="k")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [8]:
def dist_compute(particles, **kwargs):
    """
    plot the phase space of given particles
    particles can be of type Particles, list(Particles), dict{popname:Particles}

    kwargs:
    * axis : ("x", "Vx"), ("x", "Vy"), ("x", "Vz"), ("Vx", "Vy") (default) --
       ("Vx", "Vz"), ("Vy", "Vz")
    * bins :  number of bins in each dimension, default is (50,50)
    * sigma : width of the gaussian filter, default is (0,0)
    * norm  : histogram will be normed to Normalize(0,norm)
    * kde : (default False) : adds contours of kernel density estimate
    * title : (str) title of the plot
    * xlabel, ylabel
    * xlim, ylim
    * bulk : (bool) (default : False), adds vertical/horizontal lines --
             at in-plane bulk velocity for velocity axis
    * filename : (str) if exists, save plot to figure under that name

    return value : fig,ax
    """
    from pyphare.pharesee.particles import Particles, aggregate

    if isinstance(particles, list):
        particles = aggregate(particles)
    elif isinstance(particles, dict):
        particles = aggregate([p for p in particles.values()])

    if not isinstance(particles, Particles):
        raise ValueError("Error, 'particles' type should be Particles, list or dict")

    from scipy.ndimage import gaussian_filter as gf
    
    axis = kwargs.get("axis",("Vx","Vy"))
    vaxis = {"Vx":0, "Vy":1, "Vz":2}

    if axis[0] in vaxis:
        x = particles.v[:,vaxis[axis[0]]]
    elif axis[0] == "x":
        x = particles.x
    if axis[1] in vaxis:
        y = particles.v[:,vaxis[axis[1]]]

    bins = kwargs.get("bins", (50,50))
    h, xh, yh  = np.histogram2d(x, y,
              bins=kwargs.get("bins", bins),
              weights=particles.weights)

    sig = kwargs.get("sigma", (0,0))

    x_center = np.array([(xh[i] + xh[i+1])/2 for i in range(len(xh)-1)])
    y_center = np.array([(yh[i] + yh[i+1])/2 for i in range(len(yh)-1)])
    
    return x_center, y_center, h


In [9]:
import numpy as np

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, output_file, push_notebook, curdoc

from bokeh.layouts import gridplot, layout, Spacer
from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider
from bokeh.plotting import ColumnDataSource, figure, output_file, show, save

from bokeh.models import Span
from bokeh.models.glyphs import Text
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown

from bokeh.models import ColorBar, LogColorMapper, ColorMapper
# output_notebook()

In [10]:
output_notebook()

Loading BokehJS ...

In [11]:
import time

In [12]:
# importing packages.
import threading
from IPython.display import display
import ipywidgets as widgets
import time

# defining progress bar 'progress', start, stop and resume buttons
# 'start_button', 'stop_button' and 'resume_button', and horizontal
# box 'Hbox'.
start_button = widgets.Button(description="► Play")
stop_button = widgets.Button(description="❚❚ Pause")
Hbox = widgets.HBox(children=[start_button, stop_button])

# defining boolean flags 'pause' and 'resume'.
pause = False

# defining 'on_button_clicked_start()' function.
def on_button_clicked_start(b):
    # setting global variables.
    global pause
    global thread
    # conditinoal for checking whether the thread is alive;
    # if it isn't, then start it.
    if not thread.is_alive():
        thread.start()
    # else, pause and set 'restart' to True for setting
    # progress bar values to 0.
    else:
        pause = True
        time.sleep(0.1)
    # conditional for changing boolean flag 'pause'.
    if pause:
        pause = not pause
    
# defining 'on_button_clicked_stop()' function.    
def on_button_clicked_stop(b):
    # defining global variables.
    global pause
    # conditional for changing boolean flag 'pause'.
    time.sleep(0.1)
    if not pause:
        pause = not pause

# call to 'on_button_clicked_start()' function when clicking the button.
start_button.on_click(on_button_clicked_start)
# call to 'on_button_clicked_stop()' function when clicking the button.
stop_button.on_click(on_button_clicked_stop)


# defining the 'work()' function.
def work():
    # setting global variables.
    global pause
    
    while True:
        # stop/resume conditional.
        if not pause:
            # plotting annimation
            animate_update()
        time.sleep(0.1)


In [13]:
Nx=500
Ny=300

parts = get_parts(ions)
xh, yh, dist = dist_compute(parts, axis = ("x", "Vx"), color_max=0.1, bins=(Nx, Ny))
xx, yy = np.meshgrid(xh, yh)

dist[dist==0] = dist[dist>0].min()
#dist = np.log(dist)


pimage = figure(width=600,
           height=600,
           toolbar_location="below",
           tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
pimage.x_range.range_padding = pimage.y_range.range_padding = 0
# must give a vector of image data for image parameter

color_mapper = LogColorMapper(palette="Turbo256")
im = pimage.image(image=[dist.T],
        x=xh[0],
        y=yh[0],
        dw = xh[-1],
        dh=yh[-1],
        color_mapper=color_mapper,
            )
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12)

pimage.add_layout(color_bar, 'right')
pimage.title.text = "time = " + str(times[1700]) + " s (time step " + str(1700) +")"

timeslider = IntSlider(value=1700,min=0, max=len(times)-1, step=1, )
dtannimatelider = IntSlider(value=2,min=1, max=20, step=1, )

pdistrib = figure(width=300,
           height=600,
           toolbar_location="below", 
                 x_axis_type = "log")

line = pdistrib.line(x=dist.T.sum(axis=1), y = yh)



def animate_update():
    
    t = timeslider.value + dtannimatelider.value
    if t > len(times)-1:
        t = len(times)-dtannimatelider.value
    
    timeslider.value = t
    time.sleep(0.01)

    
def update_image(timestep, dt_annimate):

    ions = run.GetParticles(times[timestep], ["main", "beam"])
    parts = get_parts(ions)
    xh, yh, dist = dist_compute(parts, axis = ("x", "Vx"), color_max=0.1, bins=(Nx, Ny))
    xx, yy = np.meshgrid(xh, yh)

    dist[dist==0] = dist[dist>0].min()

    pimage.title.text = "time = " + str(times[timestep]) + " s (time step " + str(timestep) +")"
    
    im.data_source.data.update( {"image": [dist.T]})
    line.data_source.data.update( {"x":dist.T.sum(axis=1) })
    


    push_notebook(handle=handle)


In [14]:

# setting the thread.
thread = threading.Thread(target=work,)

layout = row(pimage, pdistrib)
handle = show(layout, notebook_handle=True)


interact(update_image,
         timestep=timeslider, dt_annimate=dtannimatelider)

display(Hbox)

interactive(children=(IntSlider(value=1700, description='timestep', max=1999), IntSlider(value=2, description=…